In [3]:
!python -V

Python 3.8.13


In [1]:
import pandas as pd
print(pd.__version__)

1.4.1


In [2]:
import sklearn
print(sklearn.__version__)

1.0.2


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
from collections import Counter
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
from sklearn.feature_extraction.text import CountVectorizer
import sklearn.preprocessing as preprocessing
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import category_encoders as ce
import sys  #system specific parameters and names
import gc   #garbage collector interface
import yfinance as yf
from scipy.sparse import hstack, csr_matrix


## tensorflow imports
import tensorflow as tf
from tensorflow.keras import layers 
from tensorflow.keras.regularizers import l2
print(tf.__version__)

2.8.0


## Functions for memory clean up

In [4]:
def obj_size_fmt(num):
    if num<10**3:
        return "{:.2f}{}".format(num,"B")
    elif ((num>=10**3)&(num<10**6)):
        return "{:.2f}{}".format(num/(1.024*10**3),"KB")
    elif ((num>=10**6)&(num<10**9)):
        return "{:.2f}{}".format(num/(1.024*10**6),"MB")
    else:
        return "{:.2f}{}".format(num/(1.024*10**9),"GB")

def memory_usage():
    memory_usage_by_variable=pd.DataFrame({k:sys.getsizeof(v)\
    for (k,v) in globals().items()},index=['Size'])
    memory_usage_by_variable=memory_usage_by_variable.T
    memory_usage_by_variable=memory_usage_by_variable.sort_values(by='Size',ascending=False).head(10)
    memory_usage_by_variable['Size']=memory_usage_by_variable['Size'].apply(lambda x: obj_size_fmt(x))
    return memory_usage_by_variable

In [5]:
import os
# the current working directory.
currentDirectory = os.getcwd()
print(currentDirectory)

C:\Users\dey4d\CapstoneProj


In [5]:
%%time
file_in = r'C:\Users\dey4d\CapstoneProj\index_sec'
import pickle as pickle
with open(file_in, "rb") as fh:
    data2 = pickle.load(fh)

fh.close()

CPU times: total: 13.3 s
Wall time: 14.2 s


In [6]:
data2.head(2)

CompanyCIK               CompanyName FileType   FileDate  \
FileDate                                                               
2019-01-03      717954             UNIFIRST CORP     10-Q 2019-01-03   
2019-01-03     1084765  RESOURCES CONNECTION INC     10-Q 2019-01-03   

                                           EdgarTextUrl  \
FileDate                                                  
2019-01-03   edgar/data/717954/0001284084-19-000002.txt   
2019-01-03  edgar/data/1084765/0001193125-19-001543.txt   

                                                 EdgarHtmlUrl  \
FileDate                                                        
2019-01-03  edgar/data/717954/0001284084-19-000002-index.html   
2019-01-03  edgar/data/1084765/0001193125-19-001543-index....   

                 AccessionNumber SecFileName CompanyTicker  \
FileDate                                                     
2019-01-03  0001284084-19-000002   2019-QTR1           UNF   
2019-01-03  0001193125-19-001543   2019-QTR1           RGP   

            FileDate_ClosingPrice FileDate_Plus_20  FileDate_Plus_20_Price  \
FileDate                                                                     
2019-01-03             133.860001       2019-01-23              136.509995   
2019-01-03              13.600000       2019-01-23               16.590000   

            Pct_Change_20  Share_Unit_Value_Raw  \
FileDate                                          
2019-01-03       1.979676              2.649994   
2019-01-03      21.985292              2.990000   

                                    FileName  \
FileDate                                       
2019-01-03   717954_0001284084-19-000002.txt   
2019-01-03  1084765_0001193125-19-001543.txt   

                                                       f_text  \
FileDate                                                        
2019-01-03  [various estimate, the result, timely decision...   
2019-01-03  [asc topic contract term, limited number, the ...   

            file_text_length  
FileDate                      
2019-01-03              1870  
2019-01-03              2305

In [7]:
## taking only stocks with values more than 100
data_st_plus100 = data2[data2['FileDate_ClosingPrice'] > 75]

In [8]:
# data_st_plus100.info()

In [9]:
## features besides text to be used in the model
df_non_text_features = data_st_plus100.drop(columns = ['CompanyCIK','CompanyName','EdgarTextUrl','EdgarHtmlUrl','AccessionNumber','FileDate_Plus_20','FileDate_Plus_20_Price','Share_Unit_Value_Raw','FileName','f_text'])

In [10]:
df_non_text_features = df_non_text_features.reset_index(drop=True)
df_non_text_features.head(4)

FileType   FileDate SecFileName CompanyTicker  FileDate_ClosingPrice  \
0     10-Q 2019-01-03   2019-QTR1           UNF             133.860001   
1     10-Q 2019-01-04   2019-QTR1            LW              75.580002   
2     10-Q 2019-01-08   2019-QTR1           CCF              95.870003   
3     10-Q 2019-01-08   2019-QTR1          CTAS             174.720001   

   Pct_Change_20  file_text_length  
0       1.979676              1870  
1      -5.583489              1692  
2       1.648059              2048  
3       4.733290              1408

In [11]:
%%time
## extracting the month only from file date
df_non_text_features['FileDate']=df_non_text_features.FileDate.astype('str')

FileDate = df_non_text_features['FileDate']

File_Mon = []

for pd in FileDate:
    mon = pd[5:7]
    File_Mon.append(mon)
    
# adding the confromed period month to the dataframe
df_non_text_features["File_Mon"] = File_Mon

CPU times: total: 62.5 ms
Wall time: 49.9 ms


In [12]:
%%time
## extracting the QTR only from SecFileName
df_non_text_features['SecFileName']=df_non_text_features.SecFileName.astype('str')

SecFileName = df_non_text_features['SecFileName']

QTR = []

for pd in SecFileName:
    qtr = pd[5:9]
    QTR.append(qtr)
    
# adding the confromed period month to the dataframe
df_non_text_features["QTR"] = QTR

CPU times: total: 0 ns
Wall time: 7.98 ms


In [13]:
# df_non_text_features.head(3)

In [14]:
%%time
### Converting Pct_Change_20  to boolean variable based on whether the change is exceeding 5%

per_change_exceeding = []

for index, row in df_non_text_features.iterrows():
    if row['Pct_Change_20'] > 5:     
          per_change_exceeding.append("True")
    else: 
        per_change_exceeding.append("False")
        
df_non_text_features["per_change_exceeding"] = per_change_exceeding

CPU times: total: 219 ms
Wall time: 219 ms


In [15]:
np.unique(per_change_exceeding)

array(['False', 'True'], dtype='<U5')

In [16]:
df_non_text_features.head(3)

FileType    FileDate SecFileName CompanyTicker  FileDate_ClosingPrice  \
0     10-Q  2019-01-03   2019-QTR1           UNF             133.860001   
1     10-Q  2019-01-04   2019-QTR1            LW              75.580002   
2     10-Q  2019-01-08   2019-QTR1           CCF              95.870003   

   Pct_Change_20  file_text_length File_Mon   QTR per_change_exceeding  
0       1.979676              1870       01  QTR1                False  
1      -5.583489              1692       01  QTR1                False  
2       1.648059              2048       01  QTR1                False

In [17]:
### Droping filedate, Secfilename and Pct_Change_20 from the dataframe
df_non_text_features = df_non_text_features.drop(columns = ['FileDate','SecFileName', 'Pct_Change_20'])

In [ ]:
# %%time
# ## adding sector based on tickers
# import yfinance as yf

# tickers = df_non_text_features['CompanyTicker']
# Sector = []

# for ticker in tickers: 
#     try:
#         sector = yf.Ticker(ticker).info['sector']       
#     except:
#         sector =" "
#         print(sys.exc_info()[0], "occurred.")     
#     Sector.append(sector) 

# print(len(Sector))
print(set(Sector))

<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'requests.exceptions.ChunkedEncodingError'> occurred.
<class 'req

<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> oc

<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> oc

<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> occurred.
<class 'requests.exceptions.ConnectionError'> oc

<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyErr

...

<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyErr

<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyErr

<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyErr

<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyErr

<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyError'> occurred.
<class 'KeyErr

In [32]:
# from collections import Counter
# len(Sector)
# print(set(Sector))
# Counter(Sector).keys()

In [32]:
# values, counts = np.unique(Sector, return_counts=True)
# print(values, ' ', counts)

[' ' 'Basic Materials' 'Consumer Cyclical' 'Consumer Defensive'
 'Financial Services' 'Healthcare' 'Industrials' 'Technology' 'Utilities']   [1 4 3 4 1 3 8 1 1]


In [18]:
df_non_text_features.head(3)

FileType CompanyTicker  FileDate_ClosingPrice  file_text_length File_Mon  \
0     10-Q           UNF             133.860001              1870       01   
1     10-Q            LW              75.580002              1692       01   
2     10-Q           CCF              95.870003              2048       01   

    QTR per_change_exceeding  
0  QTR1                False  
1  QTR1                False  
2  QTR1                False

In [19]:
%%time
## tf-idf vectorizer for ngram text

import pandas as pd

MAX_FEATURES = 20000 

def dummy_fun(doc):
    return doc

tfidvec = TfidfVectorizer(analyzer='word', 
                          tokenizer=dummy_fun,                      
                          preprocessor=dummy_fun,
                          token_pattern=None,
#                           max_features = MAX_FEATURES,
                          binary=True, 
                          smooth_idf=False,
                          max_df=0.80,     
                          min_df=0.005,       
                          norm= 'l1',      ## increase the sparsity
                          stop_words= None,
                          strip_accents=None,
                          use_idf=True,
                          sublinear_tf=True)

df_tfidvec = tfidvec.fit_transform(data_st_plus100["f_text"])

dfsp_tfidf = pd.DataFrame(df_tfidvec.toarray(), columns=tfidvec.get_feature_names_out())
print(dfsp_tfidf.head(3))
print('='*100)
print('Size of the tfidf Vector Matrix: ', dfsp_tfidf.shape)

   000  000s  101in  10b  10b18  10b51 plan  10b51 trading plan  10e  \
0  0.0   0.0    0.0  0.0    0.0         0.0                 0.0  0.0   
1  0.0   0.0    0.0  0.0    0.0         0.0                 0.0  0.0   
2  0.0   0.0    0.0  0.0    0.0         0.0                 0.0  0.0   

        10k  10k 10q  ...  yousc et seq  yousc section  \
0  0.000000      0.0  ...           0.0        0.00041   
1  0.000831      0.0  ...           0.0        0.00000   
2  0.000000      0.0  ...           0.0        0.00000   

   yousc section certification  yousmanage banking office  youtube  ytd  \
0                          0.0                        0.0      0.0  0.0   
1                          0.0                        0.0      0.0  0.0   
2                          0.0                        0.0      0.0  0.0   

   zero percent  zero share  zip code  zoning  
0           0.0         0.0       0.0     0.0  
1           0.0         0.0       0.0     0.0  
2           0.0         0.0      

In [20]:
df_tfidvec

<5978x40363 sparse matrix of type '<class 'numpy.float64'>'
	with 7110751 stored elements in Compressed Sparse Row format>

In [21]:
##  final data frame 
import pandas as pd
df_final = pd.concat([df_non_text_features, dfsp_tfidf], axis=1, join='inner')

In [104]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5978 entries, 0 to 5977
Columns: 40370 entries, FileType to zoning
dtypes: float64(40364), int64(1), object(5)
memory usage: 1.8+ GB


In [22]:
%%time
import pandas as pd
categorical_columns = ['FileType','CompanyTicker','File_Mon','QTR']

for column in categorical_columns:
#     print(column, ' ', df_final_2.dtypes[column])
    tempdf = pd.get_dummies(df_final[column], prefix=column)
    df_final = pd.merge(
        left=df_final,
        right=tempdf,
        left_index=True,
        right_index=True,
    )
    df_final = df_final.drop(columns=column)

CPU times: total: 11.5 s
Wall time: 11.5 s


In [23]:
%%time
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5978 entries, 0 to 5977
Columns: 41444 entries, FileDate_ClosingPrice to QTR_QTR4
dtypes: float64(40364), int64(1), object(1), uint8(1078)
memory usage: 1.8+ GB
CPU times: total: 14 s
Wall time: 23.6 s


In [107]:
%%time
del data2
del data_st_plus100
del dfsp_tfidf

gc.collect()
memory_usage()

CPU times: total: 1min 23s
Wall time: 3min 42s


Size
df_final               1.89GB
r                      1.33GB
df_final_explantory    1.31GB
X_train_img          255.70MB
df_final_col_bases   130.77MB
df_q_col_bases       130.75MB
X                    130.75MB
q                    130.75MB
X_train              104.61MB
X_test_img            63.91MB

In [32]:
# %%time
# from scipy import sparse
# df_final_comp = csr_matrix(df_final.astype(np.float))

In [24]:
## ploting the distribution of response variable 
missbeat_plot = sns.catplot(x="per_change_exceeding", kind="count",data=df_final,height=5, aspect=.5)
missbeat_plot.fig.suptitle("'per_change_exceeding' Distribution",
                  fontsize=15, fontdict={"weight": "bold"})
'''Though not perfect, it is not unbalanced '''

'Though not perfect, it is not unbalanced '

In [25]:
### QR factorization to find the orthogonal columns to have a better conditioned matrix

## DROP the response variable 

df_final_explantory = df_final.drop(columns = ['per_change_exceeding'])



In [26]:
# %%time
# from scipy import sparse
# df_final_explantory_comp = csr_matrix(df_final_explantory.astype(np.float))

In [27]:
# print("Memory utilised (bytes): ", sys.getsizeof(df_final_explantory_comp))

In [30]:
%%time
print('matrix condition:  ', np.linalg.cond(df_final_explantory, p=None))

'''it is an ill-conditioned matrix as the condition value is more than 2^16; so we need to use orthogonal bases'''

matrix condition:   1.2593810138696542e+21


## Using SVD decomposition

In [31]:
# ## SVD
# from numpy import dot, diag, exp, real, sin, cosh, tanh
# from scipy.linalg import svd, svdvals

In [32]:
# def omega_approx(beta):
#     """Return an approximate omega value for given beta. Equation (5) from Gavish 2014."""
#     return 0.56 * beta**3 - 0.95 * beta**2 + 1.82 * beta + 1.43

# df_final_explantory = df_final.drop(columns = ['per_change_exceeding'])

# # do SVD and find tau star hat
# U,sv,Vh = svd(df_final_explantory, False, lapack_driver='gesvd')
# beta = min(df_final_explantory.shape) / max(df_final_explantory.shape)
# tau = np.median(sv) * omega_approx(beta)

In [33]:
# rank = round(tau) + 1
# print('Optimal Cutoff Ranks, after adjustment for rounding, for SVD as per Gavish and Donoho : ', rank) 

In [34]:
# ### top 7 column bases represent most of the variation
# percentage_of_variation = round((sum(np.diagonal(np.diag(sv[:7]))) / sum(np.diagonal(np.diag(sv))))*100 , 2)

# print('percentage of variation represented by 7 column bases :', percentage_of_variation)

In [35]:
# ## taking top columns to create lower dimension matrix
# df_lower = U[:,:7] 
# print('Size of the lower dimension matrix---')
# print(df_lower.shape)

In [36]:
# ## weighting the column bases with the inverse of variation
# # np.diag(sv[:7])
# df_lower_new = U[:,:7] @ np.linalg.inv(np.diag(sv[:7]))
# print('Size of the new lower dimension matrix---')
# print(df_lower_new.shape)

In [37]:
# df_response = pd.DataFrame(per_change_exceeding, columns =['per_change_exceeding'])
# df_response["per_change_exceeding"] = (df_response["per_change_exceeding"] == "True").astype(int)

# print(df_response['per_change_exceeding'].value_counts())

In [38]:
# # y = df_final['per_change_exceeding']  ## this is explanatory variable

# y = np.array(df_response) ## already seperated from the data and stored in the y variable. converting to array
# X = df_lower_new
# print('Size of target vector: ', y.shape)
# print('='*50)
# print('Size of explanatory matrix: ',X.shape)

In [39]:
# ## Since the data is somewhat unbalanced, I prefer to use StratifiedKFold
# from sklearn.model_selection import StratifiedKFold

# cv = StratifiedKFold(n_splits= 5,shuffle=True,random_state=42)

# for train, test in cv.split(X,y):
#     X_train = X[train] 
#     X_test  = X[test] 
#     y_train = y[train]
#     y_test  = y[test] 
    
# print('Size of training data: ', len(X_train), 'and its shape : ', X_train.shape)
# print('Size of training labels: ', len(y_train), 'and its shape : ', y_train.shape)
# print('Size of test data: ', len(X_test), 'and its shape : ', X_test.shape)
# print('Size of test labels: ', len(y_test), 'and its shape : ', y_test.shape)

## Dense Network

In [40]:
# # defining the architecture of this connected neural network

# def build_fc_model():    
#     '''defining the model using the Sequential class'''
#     fc_model = tf.keras.Sequential([
#       # First define a input layer
#       tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),

#       # Defining the activation function for the first fully connected (Dense) layer      
#       tf.keras.layers.Dense(100, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.L2(0.01)),
        
#       # Defining the activation function for the second fully connected (Dense) layer      
#       tf.keras.layers.Dense(100, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.L2(0.01)),     

#       # Defining the activation function for the third fully connected (Dense) layer      
#       tf.keras.layers.Dense(100, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.L2(0.01)),   
        
#       # Defining the second Dense layer to output the classification probabilities
#       tf.keras.layers.Dense(1, activation=tf.nn.sigmoid, activity_regularizer=tf.keras.regularizers.L2(0.01))       
#     ])
#     return fc_model

# model = build_fc_model()
# model.summary()

In [41]:
# ## Using Adam optimizer with a learning rate of 0.01
# ## since it is categorical classification, we are opting for caterorical_crossentropy for sparse data
# ## chosing the matrix as accuracy

# learning_rate = 0.0001

# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
#               loss='binary_crossentropy',
#               metrics=['accuracy'])

In [42]:
# # Defining the batch size and the number of epochs to use during training
# BATCH_SIZE = 32
# EPOCHS = 5

# model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS)

## Using QR decomposition 

In [113]:
%%time
# del df_final
# del df_non_text_features
# del SecFileName
# del FileDate
# del QTR
# del tempdf
# del File_Mon

gc.collect()
memory_usage()

CPU times: total: 19.9 s
Wall time: 5min 4s


Size
df_final               1.89GB
df_final_explantory    1.89GB
r                      1.33GB
X_train_img          255.70MB
df_final_col_bases   130.77MB
df_q_col_bases       130.75MB
X                    130.75MB
q                    130.75MB
X_train              104.61MB
X_test_img            63.91MB

In [28]:
# %%time
# ## converting datatype from float64 to float32 to reduce the size of the matrix
# for col in df_final_explantory.columns:
#     if df_final_explantory[col].dtype == 'float64':
#         df_final_explantory[col] = df_final_explantory[col].astype('float32') 

In [29]:
# %%time
# df_final_explantory.info()

In [30]:
%%time
### QR factorization to find the orthogonal columns to have a better conditioned matrix

# df_final_explantory = df_final.drop(columns = ['per_change_exceeding'])
matrix = df_final_explantory.to_numpy()

CPU times: total: 531 ms
Wall time: 505 ms


In [31]:
%%time
# del df_final_explantory

gc.collect()
memory_usage()

CPU times: total: 14.6 s
Wall time: 1min 15s


Size
df_final                1.89GB
df_final_explantory     1.89GB
dfsp_tfidf              1.89GB
data2                 806.41MB
data_st_plus100       111.91MB
df_non_text_features    1.86MB
FileDate              391.28KB
SecFileName           385.44KB
File_Mon               47.32KB
QTR                    47.32KB

In [32]:
%%time
## qr factorization 
q, r = np.linalg.qr(matrix)

CPU times: total: 2min 15s
Wall time: 41.3 s


In [33]:
matrix.shape

(5978, 41443)

In [34]:
q.shape

(5978, 5978)

In [35]:
r.shape

(5978, 41443)

In [36]:
%%time
### putting the column bases (q) into dataframe

column_bases = [f'col_{num}' for num in range(len(np.transpose(q)))]

df_q_col_bases = pd.DataFrame(q, columns=column_bases)

df_q_col_bases.head(3)

CPU times: total: 15.6 ms
Wall time: 104 ms


col_0     col_1     col_2     col_3     col_4     col_5     col_6  \
0 -0.000126 -0.009772 -0.000233 -0.000251 -0.000501 -0.001321 -0.000393   
1 -0.000071 -0.008844 -0.000211 -0.000227 -0.000453 -0.001195 -0.000356   
2 -0.000090 -0.010704 -0.000255 -0.000275 -0.000549 -0.001447 -0.000431   

      col_7     col_8     col_9  ...      col_5968      col_5969  \
0 -0.001031 -0.000955 -0.000518  ... -3.749105e-16  2.004069e-16   
1 -0.000933 -0.000865 -0.000469  ...  2.405403e-16 -8.117232e-17   
2 -0.001130 -0.001046 -0.000567  ...  4.019094e-16 -1.557057e-15   

       col_5970      col_5971      col_5972      col_5973      col_5974  \
0  2.007522e-16 -1.007416e-16 -5.563976e-17 -7.742319e-17  1.682825e-16   
1 -3.247136e-17 -2.512010e-16 -1.621608e-17 -2.019920e-17 -2.195849e-16   
2 -1.753587e-16  1.204940e-15 -4.813111e-16  1.164407e-15  1.068122e-15   

       col_5975      col_5976      col_5977  
0 -1.630600e-16 -5.526014e-17 -2.682233e-16  
1  2.551391e-16  3.258901e-16 -8.869459e-17  
2  1.574708e-17  5.056919e-16  5.804309e-16  

[3 rows x 5978 columns]

In [37]:
## new dataframe with response variable

df_response = pd.DataFrame(per_change_exceeding, columns =['per_change_exceeding'])
df_response["per_change_exceeding"] = (df_response["per_change_exceeding"] == "True").astype(int)

df_final_col_bases = pd.concat([df_response, df_q_col_bases], axis=1, join='inner')
df_final_col_bases.head(3)    

per_change_exceeding     col_0     col_1     col_2     col_3     col_4  \
0                     0 -0.000126 -0.009772 -0.000233 -0.000251 -0.000501   
1                     0 -0.000071 -0.008844 -0.000211 -0.000227 -0.000453   
2                     0 -0.000090 -0.010704 -0.000255 -0.000275 -0.000549   

      col_5     col_6     col_7     col_8  ...      col_5968      col_5969  \
0 -0.001321 -0.000393 -0.001031 -0.000955  ... -3.749105e-16  2.004069e-16   
1 -0.001195 -0.000356 -0.000933 -0.000865  ...  2.405403e-16 -8.117232e-17   
2 -0.001447 -0.000431 -0.001130 -0.001046  ...  4.019094e-16 -1.557057e-15   

       col_5970      col_5971      col_5972      col_5973      col_5974  \
0  2.007522e-16 -1.007416e-16 -5.563976e-17 -7.742319e-17  1.682825e-16   
1 -3.247136e-17 -2.512010e-16 -1.621608e-17 -2.019920e-17 -2.195849e-16   
2 -1.753587e-16  1.204940e-15 -4.813111e-16  1.164407e-15  1.068122e-15   

       col_5975      col_5976      col_5977  
0 -1.630600e-16 -5.526014e-17 -2.682233e-16  
1  2.551391e-16  3.258901e-16 -8.869459e-17  
2  1.574708e-17  5.056919e-16  5.804309e-16  

[3 rows x 5979 columns]

In [38]:
df_response['per_change_exceeding'].value_counts()

0    4621
1    1357
Name: per_change_exceeding, dtype: int64

In [41]:
# %%time
# del df_q_col_bases
# del r
# del q

# gc.collect()
# memory_usage()

In [42]:
## X for explanatory variables and y for the response variable
## Normalize explanatory variables 
y = df_final_col_bases['per_change_exceeding']
X = df_final_col_bases.drop('per_change_exceeding', axis=1)

# from sklearn.preprocessing import Normalizer
# X = Normalizer().fit(X_raw)
print('Size of target vector: ', y.shape)
print('='*50)
print('Size of explanatory matrix: ',X.shape)

Size of target vector:  (5978,)
Size of explanatory matrix:  (5978, 5978)


In [43]:
%%time
# del df_final_col_bases

# gc.collect()
# memory_usage()

CPU times: total: 0 ns
Wall time: 0 ns


In [44]:
## Spliting data using stratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

cross_val = StratifiedKFold(n_splits=15,shuffle=True,random_state=42)

for train, test in cross_val.split(X,y):
    X_train = X.iloc[train]
    X_test  = X.iloc[test]
    y_train = y.iloc[train]
    y_test  = y.iloc[test] 

print('Size of training data: ', len(X_train), 'and its shape : ', X_train.shape)
print('Size of training labels: ', len(y_train), 'and its shape : ', y_train.shape)
print('Size of test data: ', len(X_test), 'and its shape : ', X_test.shape)
print('Size of test labels: ', len(y_test), 'and its shape : ', y_test.shape)

Size of training data:  5580 and its shape :  (5580, 5978)
Size of training labels:  5580 and its shape :  (5580,)
Size of test data:  398 and its shape :  (398, 5978)
Size of test labels:  398 and its shape :  (398,)


In [45]:
%%time
# del X

# gc.collect()
# memory_usage()

CPU times: total: 0 ns
Wall time: 0 ns


In [46]:
# Normalize all of the explanatory features with StandardScaler in train and test sets
from sklearn.preprocessing import StandardScaler
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)

In [47]:
y_train.unique()

array([0, 1])

## Dense Network

In [48]:
# defining the architecture of this connected neural network

def build_fc_model():    
    '''defining the model using the Sequential class'''
    fc_model = tf.keras.Sequential([
      # First define a input layer
      tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),

      # Defining the activation function for the first fully connected (Dense) layer      
      tf.keras.layers.Dense(500, activation=tf.nn.relu,  kernel_initializer = "HeNormal", use_bias = True, 
                           bias_initializer='HeNormal', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
        
      # Normalizing the batch to prevent from exploding  
      tf.keras.layers.BatchNormalization(),        
        
      # Adding dropout to prevent from overfitting
      tf.keras.layers.Dropout(0.5),    
        
      # Defining the activation function for the second fully connected (Dense) layer      
      tf.keras.layers.Dense(1000, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.L2(0.01)),   

      # adding dropout to prevent from overfitting 
      tf.keras.layers.Dropout(0.5),      
        
      # Defining the activation function for the third fully connected (Dense) layer      
      tf.keras.layers.Dense(500, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.L2(0.01)), 
        
      # Normalizing the batch to prevent from exploding        
      tf.keras.layers.BatchNormalization(),           
        
      # Defining the second Dense layer to output the classification probabilities
      tf.keras.layers.Dense(1, activation=tf.nn.sigmoid, activity_regularizer=tf.keras.regularizers.L2(0.01))       
    ])
    return fc_model

model = build_fc_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               2989500   
                                                                 
 batch_normalization (BatchN  (None, 500)              2000      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 500)               0         
                                                                 
 dense_1 (Dense)             (None, 1000)              501000    
                                                                 
 dropout_1 (Dropout)         (None, 1000)              0         
                                                                 
 dense_2 (Dense)             (None, 500)               500500    
                                                        

In [49]:
## Using Adam optimizer with a learning rate of 0.01
## since it is categorical classification, we are opting for caterorical_crossentropy for sparse data
## chosing the matrix as accuracy

learning_rate = 0.0001

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [50]:
# Defining the batch size and the number of epochs to use during training
BATCH_SIZE = 32
EPOCHS = 10

model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/10
175/175 [==============================] - 8s 39ms/step - loss: 22.5210 - accuracy: 0.5443
Epoch 2/10
175/175 [==============================] - 7s 41ms/step - loss: 19.6597 - accuracy: 0.6532
Epoch 3/10
175/175 [==============================] - 7s 40ms/step - loss: 17.2775 - accuracy: 0.7224
Epoch 4/10
175/175 [==============================] - 7s 42ms/step - loss: 15.1950 - accuracy: 0.7928
Epoch 5/10
175/175 [==============================] - 7s 42ms/step - loss: 13.3167 - accuracy: 0.8568
Epoch 6/10
175/175 [==============================] - 7s 42ms/step - loss: 11.6313 - accuracy: 0.9165
Epoch 7/10
175/175 [==============================] - 7s 42ms/step - loss: 10.1453 - accuracy: 0.9459
Epoch 8/10
175/175 [==============================] - 7s 39ms/step - loss: 8.7986 - accuracy: 0.9694
Epoch 9/10
175/175 [==============================] - 7s 40ms/step - loss: 7.6024 - accuracy: 0.9751
Epoch 10/10
175/175 [==============================] - 7s 40ms/step - loss: 6.5209 -

In [51]:
## Evaluating the model with the test data

test_loss, test_acc = model.evaluate(X_test, y_test) 

print('Test loss:', test_loss)
print('----------------------------')
print('Test accuracy:', test_acc)

13/13 [==============================] - 0s 10ms/step - loss: 7.1099 - accuracy: 0.7714
Test loss: 7.109867095947266
----------------------------
Test accuracy: 0.7713567614555359


## Converting images

In [52]:
## shape of the matrix
df_q_col_bases.shape

(5978, 5978)

In [64]:
## Converting images from the data row
from PIL import Image 
images = df_q_col_bases.apply(lambda x: Image.fromarray(x.values.reshape(5978,1), 'L').resize((100, 100)), axis=1)

# show image 0
images[0]

In [65]:
import matplotlib.pyplot as plt
image = images[0]
plt.imshow(image)
plt.show()

In [66]:
# images_arr = np.array(images)
## converting images (png) to tensor

imagetensor = []

for i in range(len(images)): 
    image_tensor = tf.convert_to_tensor(images[i])
    imagetensor.append(image_tensor)

In [67]:
imagetensor[20]

<tf.Tensor: shape=(100, 100), dtype=uint8, numpy=
array([[136, 136, 136, ..., 136, 136, 136],
       [120, 120, 120, ..., 120, 120, 120],
       [123, 123, 123, ..., 123, 123, 123],
       ...,
       [134, 134, 134, ..., 134, 134, 134],
       [144, 144, 144, ..., 144, 144, 144],
       [124, 124, 124, ..., 124, 124, 124]], dtype=uint8)>

In [68]:
image_arr = np.array(imagetensor)

In [69]:
image_arr[20]

array([[136, 136, 136, ..., 136, 136, 136],
       [120, 120, 120, ..., 120, 120, 120],
       [123, 123, 123, ..., 123, 123, 123],
       ...,
       [134, 134, 134, ..., 134, 134, 134],
       [144, 144, 144, ..., 144, 144, 144],
       [124, 124, 124, ..., 124, 124, 124]], dtype=uint8)

In [70]:
## finding training and testing impages
X_train_img = image_arr[train]
X_test_img = image_arr[test]

In [71]:
X_train_img[20]

array([[116, 116, 116, ..., 116, 116, 116],
       [118, 118, 118, ..., 118, 118, 118],
       [122, 122, 122, ..., 122, 122, 122],
       ...,
       [128, 128, 128, ..., 128, 128, 128],
       [130, 130, 130, ..., 130, 130, 130],
       [119, 119, 119, ..., 119, 119, 119]], dtype=uint8)

### CNN 

In [72]:
X_train_img = X_train_img / 255
X_test_img = X_test_img / 255

In [73]:
X_train_img[20]

array([[0.45490196, 0.45490196, 0.45490196, ..., 0.45490196, 0.45490196,
        0.45490196],
       [0.4627451 , 0.4627451 , 0.4627451 , ..., 0.4627451 , 0.4627451 ,
        0.4627451 ],
       [0.47843137, 0.47843137, 0.47843137, ..., 0.47843137, 0.47843137,
        0.47843137],
       ...,
       [0.50196078, 0.50196078, 0.50196078, ..., 0.50196078, 0.50196078,
        0.50196078],
       [0.50980392, 0.50980392, 0.50980392, ..., 0.50980392, 0.50980392,
        0.50980392],
       [0.46666667, 0.46666667, 0.46666667, ..., 0.46666667, 0.46666667,
        0.46666667]])

In [74]:
X_train_img.shape

(5580, 100, 100)

In [75]:
## CNN architecture 
model = tf.keras.Sequential(
    [
    tf.keras.layers.Conv2D(100, (5,5), padding='same', kernel_initializer = "HeNormal", use_bias = True, 
                           bias_initializer='HeNormal', activation="relu",input_shape=(100, 100, 1)),
    tf.keras.layers.MaxPooling2D((3, 3), strides=2),

    tf.keras.layers.Conv2D(100, (5,5), padding='same',activation="relu"),
    tf.keras.layers.MaxPooling2D((3, 3), strides=2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation="softmax")
]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 100)     2600      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 100)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 49, 49, 100)       250100    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 100)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 57600)             0         
                                                                 
 dense_4 (Dense)             (None, 100)              

In [76]:
## Using Adam optimizer with a learning rate of 0.01
## since it is categorical classification, we are opting for caterorical_crossentropy for sparse data
## chosing the matrix as accuracy

learning_rate = 0.0001

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [77]:
# Defining the batch size and the number of epochs to use during training
BATCH_SIZE = 32
EPOCHS = 5

model.fit(X_train_img, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/5
175/175 [==============================] - 178s 1s/step - loss: 0.8304 - accuracy: 0.2271
Epoch 2/5
175/175 [==============================] - 247s 1s/step - loss: 0.7359 - accuracy: 0.2271
Epoch 3/5
175/175 [==============================] - 251s 1s/step - loss: 0.6897 - accuracy: 0.2271
Epoch 4/5
175/175 [==============================] - 254s 1s/step - loss: 0.6625 - accuracy: 0.2271
Epoch 5/5
175/175 [==============================] - 247s 1s/step - loss: 0.6390 - accuracy: 0.2271


In [78]:
## Evaluating the model with the test data

test_loss, test_acc = model.evaluate(X_test_img, y_test) 

print('Test loss:', test_loss)
print('----------------------------')
print('Test accuracy:', test_acc)

13/13 [==============================] - 6s 486ms/step - loss: 0.5525 - accuracy: 0.2261
Test loss: 0.5524895191192627
----------------------------
Test accuracy: 0.2261306494474411


## Random Forest

In [59]:
## grid parameters
from pprint import pprint
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 300, num = 4)]
max_depth = [int(x) for x in np.linspace(10, 20, num = 3)]
max_depth.append(None)
min_samples_split = [5, 10, 15]
min_samples_leaf = [5, 10, 15]
max_leaf_nodes = [5, 10,15]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf, 
               'max_leaf_nodes' : max_leaf_nodes}
pprint(random_grid)

{'max_depth': [10, 15, 20, None],
 'max_leaf_nodes': [5, 10, 15],
 'min_samples_leaf': [5, 10, 15],
 'min_samples_split': [5, 10, 15],
 'n_estimators': [50, 133, 216, 300]}


In [62]:
# %%time
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.ensemble import RandomForestClassifier

# # Use the random grid to search for best hyperparameters
# clf_rf = RandomForestClassifier()

# rf_random = RandomizedSearchCV(estimator = clf_rf, param_distributions = random_grid, n_iter = 250,  cv = None, scoring = 'accuracy',  verbose=2, random_state=21, n_jobs = -1)
# # Fit the random search model
# rf_random.fit(X_train, y_train)
# pprint(rf_random.best_params_)

In [63]:
%%time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  accuracy_score

rf_clf = RandomForestClassifier(n_jobs=-1, n_estimators=500, 
                                random_state=42)
rf_clf.fit(X_train, y_train)
## Model evaluation 
predictions = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print('-------------Model Performance----------------')   
print('Accuracy = {:0.2%}.'.format(accuracy))

-------------Model Performance----------------
Accuracy = 77.39%.
CPU times: total: 19min 42s
Wall time: 2min 49s


## XG Boost

In [56]:
# !pip install xgboost
from xgboost import XGBClassifier

In [57]:
%%time
best_xgb = XGBClassifier(n_jobs=-1, 
#                             gamma= 2,
                            learning_rate=0.2, 
                            n_estimators=400,
#                             colsample_bytree=0.5,
                            max_depth=12, 
#                             min_child_weight=0.5, 
#                             subsample=0.4,
#                             objective='binary:logistic',
#                             booster = 'dart',           
#                             sample_type = 'weighted',
#                             normalize_type = 'forest',
#                             rate_drop = 0.15,
#                             skip_drop = 0.6,
                            random_state=42)

best_xgb.fit(X_train, y_train)

CPU times: total: 1h 41min 19s
Wall time: 13min 23s


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=12, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=400,
              n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, ...)

In [58]:
%%time

yhat_xgb = best_xgb.predict(X_test) ## prediction 
accuracy = accuracy_score(y_test, yhat_xgb)

print('-------------Model Performance----------------')   
print('Accuracy = {:0.2%}'.format(accuracy))


-------------Model Performance----------------
Accuracy = 77.39%
CPU times: total: 719 ms
Wall time: 120 ms
